#### Summary
- bigscience/bloom-7b1
- lora fine-tune bloom: 可插拔式的（plugin/adapter）
    - freeeze original weights
    - plugin lora adapters (peft)
- huggingface transformers 库
    - trainer.train 的参数及过程；
    - mlm 与 clm 的差异：（都是 unsupervised learning，都可以自动地构建 input/labels）
        - mlm：bert
        - clm：gpt（bloom）
    - pipeline
        - dataset/tasks
        - tokenizer
        - training (fine-tune base lora)
        - inference

#### base model & lora adapters

In [6]:
import os

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_config

In [2]:
%load_ext watermark

In [3]:
%watermark --iversions

sys         : 3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]
torch       : 2.0.0+cu118
bitsandbytes: 0.38.1



In [7]:
pretrained_model_dir = '/home/server/zhuyue/pretrained_model'
model_path = os.path.join(pretrained_model_dir, 'bloom-7b1')
model_path

'/home/server/zhuyue/pretrained_model/bloom-7b1'

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=True, device_map='auto')

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/server/anaconda3/envs/llm/lib/python3.10/site-packages/urllib3/connectionpool.py:703 in    │
│ urlopen                                                                                          │
│                                                                                                  │
│    700 │   │   │   │   self._prepare_proxy(conn)                                                 │
│    701 │   │   │                                                                                 │
│    702 │   │   │   # Make the request on the httplib connection object.                          │
│ ❱  703 │   │   │   httplib_response = self._make_request(                                        │
│    704 │   │   │   │   conn,                                                                     │
│    705 │   │   │   │   method,                                                                   │
│    706 │   │   │   │   url,                                                                      │
│                                                                                                  │
│ /home/server/anaconda3/envs/llm/lib/python3.10/site-packages/urllib3/connectionpool.py:386 in    │
│ _make_request                                                                                    │
│                                                                                                  │
│    383 │   │                                                                                     │
│    384 │   │   # Trigger any extra validation we need to do.                                     │
│    385 │   │   try:                                                                              │
│ ❱  386 │   │   │   self._validate_conn(conn)                                                     │
│    387 │   │   except (SocketTimeout, BaseSSLError) as e:                                        │
│    388 │   │   │   # Py2 raises this as a BaseSSLError, Py3 raises it as socket timeout.         │
│    389 │   │   │   self._raise_timeout(err=e, url=url, timeout_value=conn.timeout)               │
│                                                                                                  │
│ /home/server/anaconda3/envs/llm/lib/python3.10/site-packages/urllib3/connectionpool.py:1042 in   │
│ _validate_conn                                                                                   │
│                                                                                                  │
│   1039 │   │                                                                                     │
│   1040 │   │   # Force connect early to allow us to validate the connection.                     │
│   1041 │   │   if not getattr(conn, "sock", None):  # AppEngine might not have  `.sock`          │
│ ❱ 1042 │   │   │   conn.connect()                                                                │
│   1043 │   │                                                                                     │
│   1044 │   │   if not conn.is_verified:                                                          │
│   1045 │   │   │   warnings.warn(                                                                │
│                                                                                                  │
│ /home/server/anaconda3/envs/llm/lib/python3.10/site-packages/urllib3/connection.py:414 in        │
│ connect                                                                                          │
│                                                                                                  │
│   411 │   │   ):                                                                                 │
│   412 │   │   │   context.load_default_certs()                                                   │
│   413 │   │                                                

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [9]:
config = AutoConfig.from_pretrained(model_path)

In [10]:
tokenizer

BloomTokenizerFast(name_or_path='/home/server/zhuyue/pretrained_model/bloom-7b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

#### freeze original weights